In [235]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [236]:
pd.options.display.max_columns = 100

In [237]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [238]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [239]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [240]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [241]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [242]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                             Titre  \
0    3620      Tucker : l'homme et son rêve   
1    3619             Rita, Sue and Bob Too   
2    3618  Les trois mousquetaires : Milady   
3    3617   La neige tombait sur les cèdres   
4    3616             La Mégère apprivoisée   

                                 Titre alternatif Année  Durée  \
0  Tucker: The Man and His Dream (Titre original)  1988    111   
1                                             NaN  1987     90   
2                                             NaN  2023    121   
3         Snow Falling on Cedars (Titre original)  1999    127   
4        The Taming of the Shrew (Titre original)  1967    117   

  Chemin de l'affiche                        Bande-annonce  \
0          003704.jpg  www.youtube.com/watch?v=2Eyv0KkCTK8   
1          003703.jpg  www.youtube.com/watch?v=g8Ebm541gSk   
2          003702.jpg  www.youtube.com/watch?v=q_CLdbJnGIw   
3          003701.jpg  www.youtube.com/watch?v=SY6mC4I4SWk   
4         

In [243]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [244]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [245]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [246]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [247]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [248]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [249]:
display(df_director)

,id,Réalisateur(s)
0,1,Francis Ford Coppola
1,2,Alan Clarke
2,3,Martin Bourboulon
3,4,Scott Hicks
4,5,Franco Zeffirelli
...,...,...
1771,1772,Richard C. Sarafian
1772,1773,Chris Petit
1773,1774,Brett Ratner
1774,1775,Buster Keaton


In [250]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,003704.jpg,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,003703.jpg,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,003702.jpg,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,003701.jpg,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,003700.jpg,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [251]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3620
1,0,3517
2,0,3380
3,0,3364
4,0,2091
...,...,...
3894,1771,20
3895,1772,18
3896,1773,15
3897,1774,5


In [252]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id        Réalisateur(s)
0  1777  Francis Ford Coppola
1     2           Alan Clarke
2     3     Martin Bourboulon
3     4           Scott Hicks
4     5     Franco Zeffirelli
     id  Numéro
0  1777    3620
1  1777    3517
2  1777    3380
3  1777    3364
4  1777    2091


3899

In [253]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [254]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [255]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3620

# table movie, screenwriter et intermediaire

In [256]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [257]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [258]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [259]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [260]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [261]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [262]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Arnold Schulman
1,1,David Seidler
2,2,Andrea Dunbar
3,3,Alexandre de La Patellière
4,4,Matthieu Delaporte
...,...,...
3881,3881,Ted Tally
3882,3882,David Veloz
3883,3883,Richard Rutowski
3884,3884,Buster Keaton


In [263]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [264]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3620
1,0,2095
2,1,3620
3,1,2779
4,2,3619
...,...,...
7008,3881,15
7009,3882,14
7010,3883,14
7011,3884,5


In [265]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id               Scénariste(s)
0  3886             Arnold Schulman
1     1               David Seidler
2     2               Andrea Dunbar
3     3  Alexandre de La Patellière
4     4          Matthieu Delaporte
     id  Numéro
0  3886    3620
1  3886    2095
2     1    3620
3     1    2779
4     2    3619


7013

In [266]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [267]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [268]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [269]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [270]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [271]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [272]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [273]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [274]:
display(df_music)

,id,Compositeur(s)
0,0,Joe Jackson
1,1,Michael Kamen
2,2,NaN
3,3,James Newton Howard
4,4,Nino Rota
...,...,...
1272,1272,Machine Head
1273,1273,Photek
1274,1274,John Frizzell
1275,1275,Leonard Cohen


In [275]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [276]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3620
1,1,3619
2,1,2828
3,1,2664
4,1,2278
...,...,...
4006,1272,46
4007,1273,46
4008,1274,42
4009,1275,14


In [277]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id       Compositeur(s)
0  1277          Joe Jackson
1     1        Michael Kamen
2     0                  NaN
3     3  James Newton Howard
4     4            Nino Rota
     id  Numéro
0  1277    3620
1     1    3619
2     1    2828
3     1    2664
4     1    2278


4011

In [278]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [279]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4011

# table movie, studio et intermediaire

In [280]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [281]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [282]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [283]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [284]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [285]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [286]:
display(df_studio)

,id,Studio
0,0,Lucasfilm Ltd.
1,1,Umbrella Films
2,2,Constantin Film
3,3,Universal Pictures
4,4,Columbia Pictures
...,...,...
951,951,A-Film Distribution
952,952,TFC
953,953,Dybex
954,954,British Empire Films Australia


In [287]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [288]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3620
1,0,2872
2,0,2338
3,1,3619
4,2,3618
...,...,...
3617,951,172
3618,952,171
3619,953,169
3620,954,108


In [289]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


    id              Studio
0  956      Lucasfilm Ltd.
1    1      Umbrella Films
2    2     Constantin Film
3    3  Universal Pictures
4    4   Columbia Pictures
    id  Numéro
0  956    3620
1  956    2872
2  956    2338
3    1    3619
4    2    3618


3622

In [290]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [291]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3622

# table movie, casting & intermediaire

In [292]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [293]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [294]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [295]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [296]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [297]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [298]:
display(df_casting)

,id,Distribution
0,0,Jeff Bridges
1,1,Joan Allen
2,2,Martin Landau
3,3,Frederic Forrest
4,4,Mako
...,...,...
9901,9901,Paulette Goddard
9902,9902,Henry Bergman
9903,9903,Tiny Sandford
9904,9904,Jackie Coogan


In [299]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [300]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3620
1,0,2836
2,0,2332
3,0,1034
4,0,852
...,...,...
17178,9901,2
17179,9902,2
17180,9903,2
17181,9904,1


In [301]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id      Distribution
0  9906      Jeff Bridges
1     1        Joan Allen
2     2     Martin Landau
3     3  Frederic Forrest
4     4              Mako
     id  Numéro
0  9906    3620
1  9906    2836
2  9906    2332
3  9906    1034
4  9906     852


17183

In [302]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [303]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [304]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [305]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [306]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [307]:
df_country_movie  = df_country_movie.explode("Pays")

In [308]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [309]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [310]:
display(df_country)

,id,Pays
0,0,Etats-Unis
1,1,Royaume-Uni
2,2,France
3,3,Japon
4,4,Italie
5,5,Inde
6,6,Pologne
7,7,Allemagne
8,8,Danemark
9,9,IE


In [311]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [312]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3620
1,0,3617
2,0,3616
3,0,3612
4,0,3609
...,...,...
3615,42,853
3616,43,921
3617,44,769
3618,45,631


In [313]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id         Pays
0  46   Etats-Unis
1   1  Royaume-Uni
2   2       France
3   3        Japon
4   4       Italie
   id  Numéro
0  46    3620
1  46    3617
2  46    3616
3  46    3612
4  46    3609


3620

In [314]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [315]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3620

# table movie, genre & intermediaire

In [316]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [317]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [318]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [319]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [320]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [321]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [322]:
display(df_genre)

,id,Genre(s)
0,0,Drame
1,1,Biopic
2,2,Comédie
3,3,Action
4,4,Aventures
5,5,Histoire
6,6,Romantique
7,7,Court-métrage
8,8,Science-fiction
9,9,Policier


In [323]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [324]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3620
1,0,3619
2,0,3617
3,0,3615
4,0,3613
...,...,...
7952,38,920
7953,38,642
7954,38,384
7955,38,352


In [325]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id   Genre(s)
0  39      Drame
1   1     Biopic
2   2    Comédie
3   3     Action
4   4  Aventures
   id  Numéro
0  39    3620
1  39    3619
2  39    3617
3  39    3615
4  39    3613


7957

In [326]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [327]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

7957

# table movie, language & intermediaire

In [328]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [329]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [330]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [331]:
df_language_movie  = df_language_movie.explode("Langues")

In [332]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [333]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [334]:
display(df_language)

,id,Langues
0,0,Anglais
1,1,Français
2,2,Allemand
3,3,Japonais
4,4,Italien
5,5,NaN
6,6,Polonais
7,7,Arabe
8,8,Vietnamien
9,9,Espagnol


In [335]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [336]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3620
1,0,3619
2,0,3617
3,0,3616
4,0,3615
...,...,...
5100,42,1060
5101,43,1053
5102,43,930
5103,43,926


In [337]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id   Langues
0  45   Anglais
1   1  Français
2   2  Allemand
3   3  Japonais
4   4   Italien
   id  Numéro
0  45    3620
1  45    3619
2  45    3617
3  45    3616
4  45    3615


5105

In [338]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [339]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5105

# table, movie, tag & intermediaire

In [340]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [341]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [342]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [343]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [344]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [345]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [346]:
display(df_tag)

,id,Etiquettes
0,0,Voiture
1,1,NaN
2,2,Alexandre Dumas
3,3,Cape et d'épée
4,4,Shakespeare
...,...,...
203,203,Momie
204,204,viking
205,205,Réalisme soviétique
206,206,Guy Debord


In [347]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3620,Tucker : l'homme et son rêve,Tucker: The Man and His Dream (Titre original),1988,111,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2Eyv0KkCTK8,La véritable histoire de Preston Tucker.,"En 1948, le jeune ingénieur américain Preston ...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Tucker -Co...,Matroska (MKV),NaN,Fichier multimédia,4.66 GB,movie/28176,96316.0
1,3619,"Rita, Sue and Bob Too",NaN,1987,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=g8Ebm541gSk,Thatcher's Britain with her knickers down,Rita and Sue are two teenagers living on a run...,"[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Rita, Sue ...",Matroska (MKV),NaN,Fichier multimédia,4.21 GB,movie/18125,91859.0
2,3618,Les trois mousquetaires : Milady,NaN,2023,121,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=q_CLdbJnGIw,"The adventure continues, chaos ensues...",Constance Bonacieux est enlevée sous les yeux ...,[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\Les Trois ...,Matroska (MKV),NaN,Fichier multimédia,4.10 GB,movie/845111,12672620.0
3,3617,La neige tombait sur les cèdres,Snow Falling on Cedars (Titre original),1999,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=SY6mC4I4SWk,First loves last forever.,"L'ile de San Piedro, dans le Pacifique Nord, e...",[My Book]F:\#DIVX\DIVX MOVIEZ\DRAMA\La Neige T...,Matroska (MKV),NaN,Fichier multimédia,2.77 GB,movie/10219,120834.0
4,3616,La Mégère apprivoisée,The Taming of the Shrew (Titre original),1967,117,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=P7v4t0_OvrY,The motion picture they were made for!,"Italie, XVIe siècle. Petruchio, un pauvre prop...",[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\La Mégèr...,Matroska (MKV),NaN,Fichier multimédia,2.40 GB,movie/25560,61407.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3616,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3617,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3618,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [348]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3620
1,0,3538
2,0,3484
3,0,3127
4,0,3094
...,...,...
4573,206,74
4574,206,73
4575,207,17
4576,207,16


In [349]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id       Etiquettes
0  208          Voiture
1    0              NaN
2    2  Alexandre Dumas
3    3   Cape et d'épée
4    4      Shakespeare
    id  Numéro
0  208    3620
1  208    3538
2  208    3484
3  208    3127
4  208    3094


4578

In [350]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [351]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )